# Shipment Tracker API

In [1]:
from functions0_basics import get_files_path, shipments_not_delivered, get_API_details
excel_path, report_path = get_files_path("Albert")
shipments_not_delivered = shipments_not_delivered(excel_path)

from functions1_TNT_requests import make_tnt_requests
tnt_results, len_shipm_numbers = make_tnt_requests(shipments_not_delivered)

from functions2_TNT_dataframe import tnt_to_dataframe
tnt_df = tnt_to_dataframe(tnt_results,len_shipm_numbers)
tnt_df.head(50)

**150 shipments NOT DELIVERED in your file**

Status,Carrier,In Transit,Exception,Totals
0,DHL,16,0,16
1,NACEX,2,1,3
2,SEUR,1,1,2
3,TNT,127,2,129


Total TNT shipment numbers: 129


,Carrier,Client Reference,Shipment Num.,Origin Date,From (City),From (Country),To (City),To (Country),Num. of Pieces,Processing Days,Summary Code,Signatory,Carrier Code Status,Last Update (Date),Last Update (Hour),Last Location,Last Action,Exception Notification,URL
0,TNT,DSD/140232,663782937,24-11-2023,Barcelona,Spain,Krnjesevci,Serbia,1,21,In Transit,,HW,24-11-2023,21:51,Barcelona,El envío ha llegado a la ubicación de TNT,,https://www.tnt.com/express/en_gc/site/shippin...
1,TNT,DSD/142142,663787660,28-11-2023,Barcelona,Spain,Shkoder,Albania,1,17,Delivered,L HABIBI,OK,15-12-2023,16:30,Albania Others,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...
2,TNT,DSD/146902,663806545,07-12-2023,Barcelona,Spain,Lipari,Italy,1,8,In Transit,,NPD,13-12-2023,14:57,Messina,El destinatario ha solicitado una notificación...,,https://www.tnt.com/express/en_gc/site/shippin...
3,TNT,DSD/146772,663805920,07-12-2023,Barcelona,Spain,Legnago,Italy,2,7,Delivered,Bisco,OK,14-12-2023,15:11,Verona,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...
4,TNT,DSD/146956,663806783,07-12-2023,Barcelona,Spain,Marseille 16,France,1,8,Delivered,jo,OK,15-12-2023,15:36,Marseille,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...
5,TNT,DSD/146512,663804853,07-12-2023,Barcelona,Spain,Ciampino,Italy,1,6,Delivered,andreozzi,OK,13-12-2023,13:40,Roma Est,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...
6,TNT,DSD/146786,663806001,07-12-2023,Barcelona,Spain,Firenze,Italy,1,7,Delivered,marco,OK,14-12-2023,15:23,Firenze Calenzano,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...
7,TNT,DSD/146898,663806562,07-12-2023,Barcelona,Spain,Sestu,Italy,2,8,In Transit,,TR,15-12-2023,08:00,Milan Malpensa,Envío en tránsito,,https://www.tnt.com/express/en_gc/site/shippin...
8,TNT,DSD/146576,663805108,07-12-2023,Barcelona,Spain,Heillecourt,France,1,8,Exception,,MLA,08-12-2023,23:58,Nancy,"Se ha producido un retraso, pero estamos traba...",Exception Alert,https://www.tnt.com/express/en_gc/site/shippin...
9,TNT,DSD/146580,663805125,07-12-2023,Barcelona,Spain,Landriano,Italy,1,6,Delivered,Timbro,OK,13-12-2023,19:06,Zibido,Envío entregado en buen estado,,https://www.tnt.com/express/en_gc/site/shippin...


In [2]:
shipments_not_delivered

,LOGIS ID,Carrier,T&T reference,Status
205,DSD144360,NACEX,DSD144360,IN TRANSIT
526,DSD146038,NACEX,DSD146038,EXCEPTION
530,DSD145862,SEUR,DSD145862,EXCEPTION
815,DSD145944,DHL,4537224576,IN TRANSIT
816,DSD146004,DHL,5764174150,IN TRANSIT
...,...,...,...,...
1603,DSD148034,TNT,663811616,IN TRANSIT
1605,DSD148038,TNT,663811633,IN TRANSIT
1608,DSD148044,TNT,663811718,IN TRANSIT
1611,DSD148050,TNT,663811695,IN TRANSIT


In [38]:
carrier = 'DHL'

# Filter rows where the 'Carrier' column is 'DHL'
dhl_shipments = shipments_not_delivered[shipments_not_delivered['Carrier'] == carrier]

def get_chunked_dhl_urls(carrier, dhl_shipments, url, max_shipments):
    """
    Generate a list of chunked URLs for DHL shipments based on the specified max_shipments.

    Parameters:
    - carrier (str): The carrier for which URLs are being generated (e.g., 'DHL').
    - dhl_shipments (pandas.DataFrame): DataFrame containing DHL shipment data.
    - url (str): The base URL for the carrier's API.
    - max_shipments (int): The maximum number of shipments to include in each API request.

    Returns:
    - list: A list of chunked URLs for DHL shipments.
    """
    
    
    # Create a list of all shipment numbers for DHL
    dhl_shipm_list = dhl_shipments['T&T reference'].tolist()

    # Convert all shipment numbers to strings
    dhl_shipm_str_list = [str(shipm_num) for shipm_num in dhl_shipm_list]

    # Create chunks of shipment numbers based on the max_shipments for DHL
    chunked_dhl_shipm = [dhl_shipm_str_list[i:i + max_shipments] for i in range(0, len(dhl_shipm_str_list), max_shipments)]

    # Initialize an empty list to store URLs
    chunked_url_list = []

    # Construct URLs for each chunk
    for chunk in chunked_dhl_shipm:
        chunked_url = str(url + ','.join(chunk))  # Concatenate shipment numbers with commas
        chunked_url_list.append(chunked_url)

    return chunked_url_list

chunked_url_list = get_chunked_dhl_urls(carrier, dhl_shipments, url, max_shipments)
chunked_url_list

['https://api-eu.dhl.com/track/shipments?trackingNumber=4537224576,5764174150,5734665823',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=8939615123,6285974225,8939605964',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=3764749100,4884823613,4884783444',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=3746403511,9951930472,3764738865',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=3746393210,5581782161,1465781925',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=1465773864']

In [36]:
from functions1_DHL_requests import make_dhl_requests

chunked_urls = make_dhl_requests(shipments_not_delivered)

#chunked_urls


NameError: name 'chunked_url' is not defined

In [8]:
carrier = 'DHL'
url, max_shipments, API_KEY, API_SECRET, headers = get_API_details(carrier)
print(url)
print(max_shipments)
print(API_KEY)

https://api-eu.dhl.com/track/shipments?trackingNumber=
3
UBFSIW2nITZlpxJUOI0InAJF8HZHPveA


In [26]:
df = shipments_not_delivered.copy()

carrier = 'DHL'

def get_chunked_dhl_urls (carrier, df):
    
    from functions0_basics import get_API_details

    # Get API details
    url, max_shipments, API_KEY, API_SECRET, headers = get_API_details(carrier)

    # Filter rows where the 'Carrier' column is 'DHL'
    dhl_shipments = df[df['Carrier'] == carrier]

    # Create a list of all shipment numbers for DHL
    dhl_shipm_list = dhl_shipments['T&T reference'].tolist()

    # Convert all shipment numbers to strings
    dhl_shipm_str_list = [str(shipm_num) for shipm_num in dhl_shipm_list]

    # Create chunks of shipment numbers based on the max_shipments for DHL
    chunked_dhl_shipm = [dhl_shipm_str_list[i:i + max_shipments] for i in range(0, len(dhl_shipm_str_list), max_shipments)]

    # Initialize an empty list to store URLs
    chunked_url_list = []

    # Construct URLs for each chunk
    for chunk in chunked_dhl_shipm:
        chunked_url = str(url + ','.join(chunk))  # Assuming you want to concatenate shipment numbers with commas
        #print(chunked_url)
        chunked_url_list.append(chunked_url)

    return chunked_url_list

chunked_url_list = get_chunked_dhl_urls (carrier, df)
chunked_url_list

['https://api-eu.dhl.com/track/shipments?trackingNumber=4537224576,5764174150,5734665823',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=8939615123,6285974225,8939605964',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=3764749100,4884823613,4884783444',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=3746403511,9951930472,3764738865',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=3746393210,5581782161,1465781925',
 'https://api-eu.dhl.com/track/shipments?trackingNumber=1465773864']